# PyMem3DG Tutorial 2 - Ensure Mesh Quality
`Cuncheng Zhu, Christopher T. Lee`

This tutorial demonstrate the complementary functionalities of PyMem3DG to ensure the quality of mesh. The tutorial does not mean to be extensive but to provide the flavor and methods that PyMem3DG uses. The extensive documentations is hosted at https://rangamanilabucsd.github.io/Mem3DG/.

To demonstrate, we set up the system to solve the remaining problem from Tutorial 1, evolving closed spherical membrane with high curvature deformation. Again, the following integration is pre-runned. Uncomment $\texttt{fe.integrate()}$ to rerun them.

In [1]:
import pymem3dg as dg

icoFace, icoVertex = dg.getIcosphere(1, 3)
p = dg.Parameters()
p.bending.Kbc = 8.22e-5 
p.tension.Ksg = 0.1
p.osmotic.isPreferredVolume = True
p.osmotic.Kv = 0.02
p.osmotic.Vt = 0.7 * 3.14 * 4 / 3

The setup is exactly the same as the last example in tutorial 1. Without any mesh curation, the mesh will be ill-formed.

## Mesh regularization
The first strategy is to regularize the mesh, restricting the tangential movement of vertices. Regularization does not change the mesh connectivity and the number of elements in the mesh, which leads to benefit of efficiency and ease of output. The three type of regularization provided by PyMem3DG includes constraints on edge length, local triangle area, and local corner angles, adjusted by $K_{se}$, $K_{sl}$ and $K_{st}$, respectively. 

In [2]:
mP = dg.MeshProcessor()
mP.meshRegularizer.Kst = 1e-7
mP.meshRegularizer.Ksl = 1e-5
mP.meshRegularizer.Kse = 1e-7
mP.meshRegularizer.readReferenceData(icoFace, icoVertex, 0)

g = dg.System(icoFace, icoVertex, p, mP)
fe_ico = dg.Euler(f = g, dt = 1, total_time = 150000, tSave = 10000, tolerance = 0, outputDir = "output/tutorial2/ico_reg")
fe_ico.verbosity = 3
# fe_ico.integrate()

With the above regularization, the mesh looks significantly nicer, with all triangles closed to equilateral.

Free free to adjust the values of each type of regularization and attain intuition on the behavior. 

As you might observe, there are several disadvantages using the method of regularization. The behavior of simulation will depend on the strength of regularization. Local angle penalty is less restrictive and will minimally affect the underlying physics, but the resolution on high curvature region becomes very coarse. We could combine it with penalty on local area and more strongly edge length, but its influence on physics and optimization can not be neglected. In summary, how to find good balance between restriction and flexibility is not obvious. 

## Vertex shift
In addition, instead of constantly applying regularization force, one could also regularize the mesh once in a while, by "centering" the vertices (*barycenter* to be exact). This can be done by toggling the vertex shift option as follows,

In [3]:
mP.meshRegularizer.Kst = 0
mP.meshRegularizer.Ksl = 0
mP.meshRegularizer.Kse = 0
mP.meshMutator.shiftVertex = True

g = dg.System(icoFace, icoVertex, p, mP)
fe_shift = dg.Euler(f = g, dt = 1, total_time = 70000, tSave = 10000, tolerance = 0, outputDir = "output/tutorial2/ico_shift")
fe_shift.verbosity = 3
fe_shift.tProcessMesh = 10000
# fe_shift.integrate()

which also lead to a good mesh

## Mesh mutation

To resolve the challenges of regularization, PyMem3DG supports adaptive mesh by mesh mutation, including edge flip, edge split and edge collapse. Because of challenges mentioned above, mesh mutation should always be turned on when running complex simulation with large deformation. 

Notice that mesh mutation will most likely increase the computational cost and size of output files. 

The computational cost involves the time to loop through elements and decide whether conduct mesh mutation, and the subsequent overheads needed for recomputation of cached quantities. Such operation happens in the frequency of data output, therefore increasing the number of data output could better prevent the deterioration of mesh, but will increase computation.

Because of the varying mesh sizes, instead of single trajectory file using high performance *NetCDF* file, output file consists of series of snapshots in $\texttt{.ply}$ format. The detail of output files and visualization will be covered in the other tutorial. 

In [4]:
mP.meshMutator.shiftVertex = True
mP.meshMutator.flipNonDelaunay = True
mP.meshMutator.splitFat = True
mP.meshMutator.splitSkinnyDelaunay = True
mP.meshMutator.splitCurved = True
mP.meshMutator.curvTol = 0.005
mP.meshMutator.collapseSkinny = True

g = dg.System(icoFace, icoVertex, p, mP)
fe_mut = dg.Euler(f = g, dt = 1, total_time = 200000, tSave = 10000, tolerance = 0, outputDir = "output/tutorial2/ico_mut")
fe_mut.verbosity = 3
fe_mut.tProcessMesh = 10000
# fe_mut.integrate()

$\texttt{meshMutator}$ is used to specify conditions for mesh mutation, which should be specified after the instantiation of $\texttt{System}$. For details, please refer to the documentation. Mesh remains well-conditioned and well-resoluted during the simulation.

## Additional notes: 
 
### Reference mesh when mutation
Similarly, in theory we could also support specifying additional reference mesh. However, it is only necessary if the current input mesh has large deviation in total surface area from the reference mesh. Since membrane is nearly unstretchable, normally total surface area remains and self-referencing is sufficient. At the time of writing the tutorial, PyMem3DG will throw run-time error when the topology of reference mesh does not agree with the input mesh.

### Regularization + mutation
PyMem3DG does not recommend using regularization in conjunction with mesh mutation because it is most likely unnecessary to do so (as we see in previous example, vertex shift somewhat fills the role of regularization) and the behavior is not fully tested.